In [7]:
import numpy as np
import pandas as pd

In [8]:
data = pd.read_csv("full-unique-sorted (1).csv")
#print(data)
data2 = pd.get_dummies(data['class'])
data = pd.concat([data,data2], axis=1)

data.drop(["class"], axis=1, inplace=True)
data = data[["id", "year", "M1", "M2", "M3", "M4", "time", "missing"]]
normalization_len = 60
data["time"] = data["time"]/normalization_len
print(data)

data.to_csv("full-unique-sorted-preprocessed.csv", index=False)

           id  year  M1  M2  M3  M4      time  missing
0           4  2011   0   0   1   0  0.183611        0
1           4  2012   0   0   0   1  0.189167        0
2           4  2013   0   0   0   1  0.184444        0
3           4  2014   0   0   0   1  0.182778        0
4          94  2013   0   0   1   0  0.322222        0
...       ...   ...  ..  ..  ..  ..       ...      ...
19174  109267  2019   0   0   1   0  0.189722        0
19175  109270  2018   1   0   0   0  0.300556        0
19176  109270  2019   1   0   0   0  0.315278        0
19177  109301  2018   0   0   0   1  0.310278        0
19178  109301  2019   0   0   0   1  0.299722        0

[19179 rows x 8 columns]


In [9]:
# PREPROCESSING
# Arrayet blir å ha dimensjoner [Antall eksempler, Maxlengde, 6(features)]
data = pd.read_csv("full-unique-sorted-preprocessed.csv")
data.drop(["year"], axis=1, inplace=True)
print(data)
data = data.to_numpy()
x = []
y = []
n_examples = 11352
print(data)
id = 0.0
count = 0
n_unique_ids = 4334
test_ids = np.load("test_ids.npy")
val_ids = np.load("val_ids.npy")
test_ids = np.sort(test_ids)
val_ids = np.sort(val_ids)

n_training_examples = 0
n_test_examples = 0
n_val_examples = 0

test_index = 0
val_index = 0

id = 0.0
for i in range(len(data)):
    #print(int(data[i][-2]))
    if id == data[i][0] and data[i][-2] != 0.0:
        if id == test_ids[test_index]:
            n_test_examples += 1
        elif id == val_ids[val_index]:
            n_val_examples += 1
        else:
            n_training_examples += 1
    else:
        id = data[i][0]
        if test_ids[test_index] < id and test_index < 399:
            test_index += 1
        if val_ids[val_index] < id and val_index < 399:
            val_index += 1

print(n_test_examples)
print(n_val_examples)
print(n_training_examples)
print(data.shape)

x_train = np.zeros((9311, 8, 6))
x_train_pads = np.zeros(9311)
x_val = np.zeros((996, 8, 6))
x_val_pads = np.zeros(996)
x_test = np.zeros((1048, 8, 6))
x_test_pads = np.zeros(1048)

y_train = np.zeros(9311)
y_val = np.zeros(996)
y_test = np.zeros(1048)

val_index = 0
test_index = 0

example_n = 0
i = 0
while i < 19179:
    if data[i][0] > test_ids[test_index] and test_index < 399:
        test_index += 1
    if data[i][0] > val_ids[val_index] and val_index < 399:
        val_index += 1
    if data[i][0] != test_ids[test_index] and data[i][0] != val_ids[val_index]:
        j = i+1
        while j < len(data) and data[i][0] == data[j][0]:
            if data[j][-1] == 0.0:
                x_train[example_n,:j-i] = data[i:j,1:]
                y_train[example_n] = data[j,-2]
                x_train_pads[example_n] = j-i
                example_n += 1
            j += 1
        i = j-1
    i += 1

y_train *= 60
print(x_train[9310])
print(y_train[9310])
print(x_train_pads[9310])
np.save("x_train", x_train)
np.save("y_train", y_train)
np.save("x_train_pads", x_train_pads)

val_index = 0
test_index = 0

example_n = 0
i = 0
while i < 19179:
    if data[i][0] > val_ids[val_index] and val_index < 399:
        val_index += 1
    if data[i][0] == val_ids[val_index]:
        j = i+1
        while j < len(data) and data[i][0] == data[j][0]:
            if data[j][-1] == 0.0:
                x_val[example_n,:j-i] = data[i:j,1:]
                y_val[example_n] = data[j,-2]
                x_val_pads[example_n] = j-i
                example_n += 1
            j += 1
        i = j-1
    i += 1

y_val *= 60
print(x_val[995])
print(y_val[995])
print(x_val_pads[995])
np.save("x_val", x_val)
np.save("y_val", y_val)
np.save("x_val_pads", x_val_pads)

val_index = 0
test_index = 0

example_n = 0
i = 0
while i < 19179:
    if data[i][0] > test_ids[test_index] and test_index < 399:
        test_index += 1
    if data[i][0] == test_ids[test_index]:
        j = i+1
        while j < len(data) and data[i][0] == data[j][0]:
            if data[j][-1] == 0.0:
                x_test[example_n,:j-i] = data[i:j,1:]
                y_test[example_n] = data[j,-2]
                x_test_pads[example_n] = j-i
                example_n += 1
            j += 1
        i = j-1
    i += 1

y_test *= 60
print(x_test[1047])
print(y_test[1047])
print(x_test_pads[1047])
np.save("x_test", x_test)
np.save("y_test", y_test)
np.save("x_test_pads", x_test_pads)

           id  M1  M2  M3  M4      time  missing
0           4   0   0   1   0  0.183611        0
1           4   0   0   0   1  0.189167        0
2           4   0   0   0   1  0.184444        0
3           4   0   0   0   1  0.182778        0
4          94   0   0   1   0  0.322222        0
...       ...  ..  ..  ..  ..       ...      ...
19174  109267   0   0   1   0  0.189722        0
19175  109270   1   0   0   0  0.300556        0
19176  109270   1   0   0   0  0.315278        0
19177  109301   0   0   0   1  0.310278        0
19178  109301   0   0   0   1  0.299722        0

[19179 rows x 7 columns]
[[4.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  1.83611111e-01 0.00000000e+00]
 [4.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.00000000e+00
  1.89166667e-01 0.00000000e+00]
 [4.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.00000000e+00
  1.84444444e-01 0.00000000e+00]
 ...
 [1.09270000e+05 1.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  3.15277778e-01 0